In [2]:
import pandas as pd
from datetime import date
from bs4 import BeautifulSoup
import requests
from playwright.async_api import async_playwright
import time
import numpy as np
pd.set_option('display.max_colwidth', None)

In [3]:
# Take today's date and format it - must be used in the URLs.

date = date.today()

date = date.strftime("%d.%m.%Y")

print(date)

02.02.2023


In [4]:
# Read in the links and update with today's date
df = pd.read_csv("data/links.csv")

df = df.replace(to_replace=r'01.01.2023', value= date, regex=True)

In [5]:
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless = True)
page = await browser.new_page()

url = df.iloc[0]['link']
await page.goto(url)

time.sleep(1)
await page.locator("#close-button").click()

In [6]:
# Create an empty dataframe for the scrape.
total_df = pd.DataFrame()

In [7]:
for index, row in df.iterrows():
    url = row['link']

    country = row['country']
    area = row['area']
    
    await page.goto(url)
    time.sleep(5)
    await page.get_by_text("PT60M").click()

    time.sleep(5)

    tables = pd.read_html(await page.content(), flavor = 'html5lib')
    data = tables[0]
    data.insert(0, 'country', country)
    data.insert(1, 'area', area)
    total_df = pd.concat([total_df, data], axis=0)

In [13]:
total_df = total_df.droplevel(0, axis=1)

ValueError: Cannot remove 1 levels from an index with 1 levels: at least one level must be left.

In [14]:
total_df.columns = ['country', 'area', 'mtu', 'currency/mwh', 'eur/mwh', 'uah/mwh', 'date']

total_df

,country,area,mtu,currency/mwh,eur/mwh,uah/mwh,date
0,Albania,BZN AL,00:00 - 01:00,-,NaN,NaN,02/02/2023
1,Albania,BZN AL,01:00 - 02:00,-,NaN,NaN,02/02/2023
2,Albania,BZN AL,02:00 - 03:00,-,NaN,NaN,02/02/2023
3,Albania,BZN AL,03:00 - 04:00,-,NaN,NaN,02/02/2023
4,Albania,BZN AL,04:00 - 05:00,-,NaN,NaN,02/02/2023
...,...,...,...,...,...,...,...
19,United Kingdom,BZN GB(IFA2),19:00 - 20:00,n/e,NaN,NaN,02/02/2023
20,United Kingdom,BZN GB(IFA2),20:00 - 21:00,n/e,NaN,NaN,02/02/2023
21,United Kingdom,BZN GB(IFA2),21:00 - 22:00,n/e,NaN,NaN,02/02/2023
22,United Kingdom,BZN GB(IFA2),22:00 - 23:00,n/e,NaN,NaN,02/02/2023


In [17]:
total_df['date'] = pd.to_datetime('today').strftime("%m/%d/%Y")

total_df = total_df[['country', 'area', 'mtu', 'currency/mwh', 'eur/mwh', 'date']]
total_df

/var/folders/hx/m3n3wwr91yg_snnv68ynqsp80000gn/T/ipykernel_49821/2277366585.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total_df['date'] = pd.to_datetime('today').strftime("%m/%d/%Y")


,country,area,mtu,currency/mwh,eur/mwh,date
0,Albania,BZN AL,00:00 - 01:00,-,NaN,02/02/2023
1,Albania,BZN AL,01:00 - 02:00,-,NaN,02/02/2023
2,Albania,BZN AL,02:00 - 03:00,-,NaN,02/02/2023
3,Albania,BZN AL,03:00 - 04:00,-,NaN,02/02/2023
4,Albania,BZN AL,04:00 - 05:00,-,NaN,02/02/2023
...,...,...,...,...,...,...
19,United Kingdom,BZN GB(IFA2),19:00 - 20:00,n/e,NaN,02/02/2023
20,United Kingdom,BZN GB(IFA2),20:00 - 21:00,n/e,NaN,02/02/2023
21,United Kingdom,BZN GB(IFA2),21:00 - 22:00,n/e,NaN,02/02/2023
22,United Kingdom,BZN GB(IFA2),22:00 - 23:00,n/e,NaN,02/02/2023


In [20]:
# Standardize the missing values
total_df.replace({'-': np.nan}, inplace=True)
total_df.replace({'n/e': np.nan}, inplace=True)

# And reorder the columns
all_data = total_df[['date', 'country', 'area', 'mtu', 'currency/mwh', 'eur/mwh']]
all_data

,date,country,area,mtu,currency/mwh,eur/mwh
0,02/02/2023,Albania,BZN AL,00:00 - 01:00,NaN,NaN
1,02/02/2023,Albania,BZN AL,01:00 - 02:00,NaN,NaN
2,02/02/2023,Albania,BZN AL,02:00 - 03:00,NaN,NaN
3,02/02/2023,Albania,BZN AL,03:00 - 04:00,NaN,NaN
4,02/02/2023,Albania,BZN AL,04:00 - 05:00,NaN,NaN
...,...,...,...,...,...,...
19,02/02/2023,United Kingdom,BZN GB(IFA2),19:00 - 20:00,NaN,NaN
20,02/02/2023,United Kingdom,BZN GB(IFA2),20:00 - 21:00,NaN,NaN
21,02/02/2023,United Kingdom,BZN GB(IFA2),21:00 - 22:00,NaN,NaN
22,02/02/2023,United Kingdom,BZN GB(IFA2),22:00 - 23:00,NaN,NaN


In [ ]:
all_data.to_csv("data/europaenergy_today.csv", index=False)